In [1]:
import torch
from torch import nn as nn
from models.nac import NAC
from models.nalu import NALU
from utilities.train_utils import get_batches
from utilities.train_utils import get_eval_loss
from utilities.train_utils import get_eval_preds
from utilities.train_utils import train
from torch.optim import Adam
import numpy as np

In [2]:
def make_data(min_val, max_val, num_obs, op):
    data = np.random.uniform(min_val, max_val, size=(num_obs, 2))
    if op == '+':
        targets = data[:, 0] + data[:, 1]
    elif op == '-':
        targets = data[:, 0] - data[:, 1]
    elif op == '*':
        targets = data[:, 0] * data[:, 1]
    elif op == '/':
        targets = data[:, 0] / data[:, 1]
    elif op == 'sqrt':
        data = np.random.uniform(min_val, max_val, size=(num_obs, 1))
        targets = np.sqrt(data)
    return data, targets

In [3]:
def accuracy_score(preds, targets, tol=1e-3):
    preds = preds.cpu().numpy().flatten()
    targets = targets.cpu().numpy().flatten()
    accuracy = np.isclose(preds, targets, rtol=tol)
    accuracy = accuracy.astype(np.int32).mean()
    return accuracy * 100

In [4]:
# def train(model, criterion, train_data, train_targets,
#           valid_data, valid_targets, patience=15, batch_size=32,
#           num_epochs=10000, checkpoint='best_model.sav'):
#     running_patience = patience
#     running_batch = 0
#     running_loss = 0
#     min_loss = float('inf')
#     use_gpu = torch.cuda.is_available()
#     if use_gpu:
#         model = model.cuda()
#     optimizer = Adam(model.parameters())
    
#     for epoch in range(1, num_epochs + 1):
#         model.train()
#         total = int(np.ceil(train_data.shape[0]/batch_size))
#         with tqdm(total=total,
#                            desc="Epoch: {}".format(epoch),
#                            leave=False) as prog_bar:

#             for x, y in get_batches(train_data, train_targets, batch_size,
#                                     mode='train', use_gpu=use_gpu):
#                 output = model(x)
#                 loss = criterion(output, y)
#                 optimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()
#                 running_loss += loss.item()
#                 running_batch += 1
#                 prog_bar.set_description_str('Training loss after {} batches: {:.3f}'.format(
#                             running_batch, running_loss/running_batch))
#                 prog_bar.update(1)
#         valid_loss = get_eval_loss(model, criterion, valid_data,
#                                    valid_targets, batch_size, False)
# #         tqdm.write("Validation loss after epoch {}: {}".format(epoch, valid_loss))
#         if valid_loss < min_loss:
#             min_loss = valid_loss
# #             tqdm.write('Validation loss improved! Saving model.')
#             with open(checkpoint, 'wb') as f:
#                 torch.save(model.state_dict(), f)
#                 running_patience = patience
#         else:
#             running_patience -= 1
#         if running_patience == 0:
#             tqdm.write('Ran out of patience, early stopping employed!')
#             break
#     model.load_state_dict(torch.load(checkpoint))
#     return model

In [5]:
eps = 1e-12

ops = ['+', '-', '*', '/', 'sqrt']
test_scores = {}
for op in ops:
    X_train, Y_train = make_data(10, 20, 1000, op)
    X_valid, Y_valid = make_data(10, 20, 100, op)
    X_test, Y_test = make_data(0, 30, 100, op)
     
    model = train(NALU(2, 1), nn.SmoothL1Loss(), Adam, X_train, Y_train,
                  X_valid, Y_valid, patience=15, batch_size=32)
    test_preds, test_targets = get_eval_preds(model, X_test, Y_test,
                                              32, False)
    test_acc = accuracy_score(test_preds, test_targets)
    print('Test accuracy: {:.2f}'.format(test_acc))
    test_scores[op] = test_acc

KeyboardInterrupt: 